# Requirements

# Imports

In [29]:
import internetarchive
import os
import tarfile
import csv
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
import configparser
import json

In [30]:
import warnings

# Suppressing the warnings
warnings.filterwarnings('ignore') 

# Functions

## General

In [1]:
#if you want to use a path that it isn't in the same folder you started your notebook, put the path
#in the option complete_path and the folder will be create there

def check_folder(name_folder, complete_path = False):            
    #Creating folder if that doesn't exist
    if local == False:
        p = %pwd
    else:
        p = local
    p = p + f'/{name_folder}'
    path = os.path.expanduser(p)
    if not os.path.exists(path):
        os.makedirs(path)
        print("{} created.".format(path))
        
    return path

In [32]:
def get_file_name(item_name, ext='*tar'):
    
    file_names = [f.name for f in internetarchive.get_files(item_name, glob_pattern= ext)]
    
    return file_names

In [98]:
def humanize(size_bytes):
    KB = 1 << 10
    MB = 1 << 20
    GB = 1 << 30

    if size_bytes < KB:
        return '{} B'.format(size_bytes)
    elif size_bytes < MB:
        return '{:.2f} KiB'.format(size_bytes/KB)
    elif size_bytes < GB:
        return '{:.2f} MiB'.format(size_bytes/MB)
    else:
        return '{:.2f} GiB'.format(size_bytes/GB)


## Cleaning and Transfomations

In [34]:
def get_data(file_path, file_name, word_bag = False, lang = False, db_update = False):
    
    #reading json
    df = pd.read_json(file_path, lines=True, compression='gzip')
    
    #updating total tweets
    if not update_db == False:
        update_db(table = 'control', field = f"count_total = '{len(df)}'", condition = f"name = '{file_name}'")
    
    #Filtering language if a language was sent
    if not lang == False:
        df = df[df.lang == lang]
    
    #taking away columns unnecessary
    df = df.loc[:, ['created_at', 'text', 'entities']]
    
    #filtering Tweets using subject word bag if a word bag was sent
    if not word_bag == False:
        bow = '|'.join(word_bag) # bag of word
        df = df[df['text'].str.contains(bow, case=False)]
        
        #updating filtered tweets
        if not update_db == False:
            update_db(table = 'control', field = f"count_filtered = '{len(df)}'", condition = f"name = '{file_name}'")
    
    return df

## Control Operations

### CSV

In [35]:
def create_control_csv(control_path, file_name = 'control.csv', fieldnames=['name','datetime', 'type', 'size' 'count_total', 'count_filtered']):
    
    control_file = f"{control_path}/{file_name}"
    
    #if file doesn't exist I'll create it
    if not os.path.exists(control_file):
        with open(control_file, 'w', newline='') as control_csv:
            writer = csv.DictWriter(control_csv, fieldnames = fieldnames)
            writer.writeheader()
        control_csv.close()
        print("{} created.".format(control_file))
    
    return control_file
    


In [ ]:
def read_control():
    
    #Reading the control file
    with open(control_file, 'r', newline='') as control_csv:
        reader = csv.DictReader(control_csv)
        reader_data = [r for r in reader]
        control_csv.seek(0)
    control_csv.close()

    return reader_data

In [36]:
def download_file_csv(item_name, file_name, download_folder = 'Downloads'):
    
    path = check_folder(download_folder)
    
    #calling function for getting control file
    control_path = path + '/control_download.csv'    
    reader_data = control_op_csv(control_path)
    
    #Getting Itens names
    item = internetarchive.get_item(item_name)
        
    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_download:
        writer = csv.DictWriter(control_download, fieldnames=['name','datetime'])
        
        #check if the file has been downloaded before
        if not any(row['name'] == file_name for row in reader_data):
            
            #downloading
            r = item.download(
                destdir=path,  # The directory to download files to
                ignore_existing=True,  # Skip files that already exist locally
                checksum=True,  # Skip files based on checksum
                verbose=True,  # Print progress to stdout
                retries=100,  # The number of times to retry on failed requests
                no_directory=True,  # Download withtout the identifier
                files = file_name)
            
            #Adding file name on control list
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
    control_download.close()
    
    path_file = path + f'/{file_name}'
    
    return path_file

In [37]:
def tar_file_csv(tar_file, file_name, extration_folder='Extraction'):

    path = check_folder(extration_folder)
    
    #calling function for getting control file
    control_path = path + '/control_extraction.csv'    
    reader_data = control_op(control_path)

    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_tar:
        writer = csv.DictWriter(control_tar, fieldnames=['name','datetime'])
        
        #check if the file has been extrated before
        if not any(row['name'] == file_name for row in reader_data):

            #Extracting file
            tar_file.extract(file_name, path=path)

            #Saving files name on control file
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
            path_file = path + f'/{file_name}'
            
        else:
            
            path_file = False
    
    control_tar.close()
    
    
    return path_file

# Code

In [ ]:
item_name = 'archiveteam-twitter-stream-2022-10'

#Getting Itens names
file_names = get_file_name(item_name = item_name)#, ext = '*jpg')

x = 0 #for testing 

for dlf in tqdm(file_names, desc='Download progress'):
    
    #if x < 1: #for testing 
    #downloading the file
    file = download_file_csv(item_name = item_name, file_name = dlf)
    
    if not file == False:

        #Open tar file
        tar = tarfile.open(file, "r")

        #Getting json files' names
        tar_file_names = list(filter(lambda t: t.endswith('.json.gz'), tar.getnames()))

        for tf in tqdm(tar_file_names, desc='Unzip progress'):

            #call funcition to extract files
            js = tar_file_csv(tar_file = tar, file_name = tf)

            #if the file has already been unzipped that part will be skipped
            if not js == False:

                #Getting, cleaning and storing Data
                bow = ['vaccination', 'vaccines', 'vaccine']
                df = get_data(file_path = js, file_name = tf, lang = 'en', word_bag = bow, db_update = True)

                if not df.empty:

                    conn = open_connection()
                    cursor = conn.cursor()

                    for index, row in df.iterrows():
                        created_at = row['created_at']
                        text = row['text']
                        entities = json.dumps(row['entities'])

                        sql = "INSERT INTO tweets (created_at, text, entities) VALUES (%s, %s, %s)"
                        cursor.execute(sql, (created_at, text, entities))

                        conn.commit()

                    cursor.close()
                    conn.close()

                #deleting json file after processing
                os.remove(js)

        #closing tar file
        tar.close()

        #x =+ 1

        #removing file downloaded
        os.remove(file)
        
    #